In [1]:
import os
import argparse
import numpy as np
import torch
import torch.backends.cudnn as cudnn
import time
from progress.bar import Bar
import torchvision.transforms as transforms
from dataloader.EyeQ_loader import DatasetGenerator
from utils.trainer import train_step, validation_step, save_output
from utils.metric import compute_metric

import pandas as pd
from networks.densenet_mcf import dense121_mcs

os.environ["CUDA_VISIBLE_DEVICES"] = "0"
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

np.random.seed(0)

/usr/lib/python3/dist-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.17.3 and <1.25.0 is required for this version of SciPy (detected version 1.26.0
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


Metric.py

In [33]:
import numpy as np
from sklearn.metrics import f1_score, confusion_matrix, roc_curve, roc_auc_score, accuracy_score

class MetricCalculator:
    def __init__(self, df, predictions, target_names):
        self.df = df
        self.predictions = predictions
        self.target_names = target_names
        self.n_class = len(target_names)
        self.argmaxPRED = np.argmax(predictions, axis=1)
        self.metrics = None

    def calculate_metrics(self):
        Accuracy_score = accuracy_score(self.df, self.argmaxPRED)
        metrics = {
            'class_name': self.target_names,
            'F1': [],
            'AUC': 0,
            'Accuracy': Accuracy_score,
            'Sensitivity': [],
            'Precision': [],
            'Specificity': [],
            'ROC_curve': {},
            'tp': [],
            'tn': [],
            'fp': [],
            'fn': [],
        }

        for i in range(self.n_class):
            tmp_label = self.df == i
            tmp_pred = self.argmaxPRED == i
            F1 = f1_score(tmp_label, tmp_pred)
            tn, fp, fn, tp = confusion_matrix(tmp_label, tmp_pred).ravel()
            outAUROC = roc_auc_score(tmp_label, self.predictions[:, i])

            metrics['F1'].append(F1)
            metrics['tp'].append(tp)
            metrics['tn'].append(tn)
            metrics['fp'].append(fp)
            metrics['fn'].append(fn)

            metrics['ROC_curve']['ROC_fpr_' + str(i)] = roc_curve(tmp_label, self.predictions[:, i])[0]
            metrics['ROC_curve']['ROC_tpr_' + str(i)] = roc_curve(tmp_label, self.predictions[:, i])[1]
            metrics['ROC_curve']['ROC_T_' + str(i)] = roc_curve(tmp_label, self.predictions[:, i])[2]
            metrics['ROC_curve']['AUC_' + str(i)] = outAUROC

            metrics['AUC'] += outAUROC
            metrics['Sensitivity'].append(tp / (tp + fn))
            metrics['Precision'].append(tp / (tp + fp))
            metrics['Specificity'].append(tn / (fp + tn))

        mPrecision = sum(metrics['tp']) / sum(metrics['tp'] + metrics['fp'])
        mRecall = sum(metrics['tp']) / sum(metrics['tp'] + metrics['fn'])

        metrics['micro-Precision'] = mPrecision
        metrics['micro-Sensitivity'] = mRecall
        metrics['micro-Specificity'] = sum(metrics['tn']) / sum(metrics['fp'] + metrics['tn'])
        metrics['micro-F1'] = 2 * mPrecision * mRecall / (mPrecision + mRecall)
        metrics['AUC'] /= self.n_class

        self.metrics = metrics

    def get_metrics(self):
        if self.metrics is None:
            self.calculate_metrics()
        return self.metrics


trainer.py Trainer class

In [45]:
class Trainer:
    def __init__(self, model, optimizer, criterion, loss_weights, epochs):
        self.model = model
        self.optimizer = optimizer
        self.criterion = criterion
        self.loss_weights = loss_weights
        self.epochs = epochs

    def train_step(self, train_loader, epoch):
        self.model.train()
        epoch_loss = 0.0
        iters_per_epoch = len(train_loader)
        print('\n\niters_per_epoch: ', iters_per_epoch, '\n\n')
        bar = Bar('Processing {} Epoch -> {} / {}'.format('train', epoch+1, self.epochs), max=iters_per_epoch)
        bar.check_tty = False

        for step, (imagesA, imagesB, imagesC, labels) in enumerate(train_loader):
            start_time = time.time()

            torch.set_grad_enabled(True)

            imagesA = imagesA.cuda()
            imagesB = imagesB.cuda()
            imagesC = imagesC.cuda()

            labels = labels.cuda()
            labels = labels.squeeze(1)

            out_A, out_B, out_C, out_F, combine = self.model(imagesA, imagesB, imagesC)

            loss_x = self.criterion(out_A, labels)
            loss_y = self.criterion(out_B, labels)
            loss_z = self.criterion(out_C, labels)
            loss_c = self.criterion(out_F, labels)
            loss_f = self.criterion(combine, labels)

            lossValue = (
                self.loss_weights[0] * loss_x +
                self.loss_weights[1] * loss_y +
                self.loss_weights[2] * loss_z +
                self.loss_weights[3] * loss_c +
                self.loss_weights[4] * loss_f
            )

            self.optimizer.zero_grad()
            lossValue.backward()
            self.optimizer.step()

            epoch_loss += lossValue.item()
            end_time = time.time()
            batch_time = end_time - start_time

            bar_str = '{} / {} | Time: {batch_time:.2f} mins | Loss: {loss:.4f} '
            bar.suffix = bar_str.format(
                step + 1, iters_per_epoch, batch_time=batch_time * (iters_per_epoch - step) / 60,
                loss=lossValue.item()
            )
            bar.next()

        epoch_loss = epoch_loss / iters_per_epoch
        bar.finish()
        return epoch_loss


trainer.py Validator class

In [38]:
class Validator:
    def __init__(self, model, criterion):
        self.model = model
        self.criterion = criterion

    def validate(self, val_loader):
        self.model.eval()
        epoch_loss = 0
        iters_per_epoch = len(val_loader)
        bar = Bar('Processing {}'.format('validation'), max=iters_per_epoch)

        for step, (imagesA, imagesB, imagesC, labels) in enumerate(val_loader):
            start_time = time.time()

            imagesA = imagesA.cuda()
            imagesB = imagesB.cuda()
            imagesC = imagesC.cuda()

            labels = labels.cuda()
            labels = labels.squeeze(1)

            _, _, _, _, outputs = self.model(imagesA, imagesB, imagesC)
            with torch.no_grad():
                loss = self.criterion(outputs, labels)
                epoch_loss += loss.item()

            end_time = time.time()

            batch_time = end_time - start_time
            bar_str = '{} / {} | Time: {batch_time:.2f} mins'
            bar.suffix = bar_str.format(step + 1, len(val_loader), batch_time=batch_time * (iters_per_epoch - step) / 60)
            bar.next()

        epoch_loss = epoch_loss / iters_per_epoch
        bar.finish()
        return epoch_loss

trainer.py Saver class ... maybe move to another file

In [35]:
class Saver:
    def __init__(self, label_list):
        self.label_list = label_list
        self.n_class = len(label_list)

    def save(self, label_test_file, dataPRED, save_file):
        datanpPRED = np.squeeze(dataPRED.cpu().numpy())
        df_tmp = pd.read_csv(label_test_file)
        image_names = df_tmp["image"].tolist()

        result = {self.label_list[i]: datanpPRED[:, i] for i in range(self.n_class)}
        result['image_name'] = image_names
        out_df = pd.DataFrame(result)

        name_order = ['image_name'] + self.label_list
        out_df.to_csv(save_file, columns=name_order)

In [2]:
# train_images_dir = data_root + '/train'
n_classes = 3
pre_model='DenseNet121_v3_v1'
lr=0.01
batch_size=4

label_train_file = '../data/Label_EyeQ_test.csv' # swapped
# test_images_dir = data_root + '/test'
label_test_file = '../data/Label_EyeQ_train.csv' # swapped


save_file_name = 'DenseNet121_v3_v1_mine.csv'

best_metric = np.inf
best_iter = 0
# options
cudnn.benchmark = True

model = dense121_mcs(n_classes)

if pre_model is not None:
    loaded_model = torch.load(os.path.join('../',pre_model + '.tar'))
    model.load_state_dict(loaded_model['state_dict'])

model.to(device)

criterion = torch.nn.BCELoss(reduction='mean')
optimizer = torch.optim.SGD(model.parameters(), lr=lr)

print('Total params: %.2fM' % (sum(p.numel() for p in model.parameters()) / 1000000.0))

/home/tomas/.local/lib/python3.10/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/tomas/.local/lib/python3.10/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)


Total params: 28.86M


In [12]:

transform_list1 = transforms.Compose([
        transforms.Resize(256),
        transforms.RandomResizedCrop(224),
        transforms.RandomHorizontalFlip(),
        transforms.RandomVerticalFlip(),
        transforms.RandomRotation(degrees=(-180, +180)),
    ])

transformList2 = transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406],
                             [0.229, 0.224, 0.225])
    ])

transform_list_val1 = transforms.Compose([
        transforms.Resize(224),
        transforms.CenterCrop(224),
    ])

In [13]:
test_images_dir ='../../preprocess/train_preprocessed'
train_images_dir = '../../preprocess/train_preprocessed'

data_test = DatasetGenerator(data_dir=test_images_dir, list_file=label_test_file, transform1=transform_list_val1,
                             transform2=transformList2, n_class=n_classes, set_name='test')


In [14]:
data_test.image_names


['../../preprocess/train_preprocessed/10009_left.png',
 '../../preprocess/train_preprocessed/10009_right.png',
 '../../preprocess/train_preprocessed/10014_left.png',
 '../../preprocess/train_preprocessed/10014_right.png',
 '../../preprocess/train_preprocessed/10015_left.png',
 '../../preprocess/train_preprocessed/10015_right.png',
 '../../preprocess/train_preprocessed/10017_left.png',
 '../../preprocess/train_preprocessed/1002_left.png',
 '../../preprocess/train_preprocessed/10022_right.png',
 '../../preprocess/train_preprocessed/10028_left.png',
 '../../preprocess/train_preprocessed/10029_left.png',
 '../../preprocess/train_preprocessed/10030_left.png',
 '../../preprocess/train_preprocessed/10030_right.png',
 '../../preprocess/train_preprocessed/10031_left.png',
 '../../preprocess/train_preprocessed/10031_right.png',
 '../../preprocess/train_preprocessed/10043_left.png',
 '../../preprocess/train_preprocessed/10043_right.png',
 '../../preprocess/train_preprocessed/10058_left.png',
 '..

In [15]:
test_loader = torch.utils.data.DataLoader(dataset=data_test, batch_size=batch_size,
                                          shuffle=False, num_workers=4, pin_memory=True)

In [16]:
print('Test loader length: ', len(test_loader.dataset.image_names))
print('Test loader length: ', len(test_loader.dataset.labels))



Test loader length:  12543
Test loader length:  12543


In [8]:
outPRED_mcs = torch.FloatTensor().cuda()
model.eval()
iters_per_epoch = len(test_loader)
bar = Bar('Processing {}'.format('inference'), max=len(test_loader))
bar.check_tty = False
for epochID, (imagesA, imagesB, imagesC) in enumerate(test_loader):
    imagesA = imagesA.cuda()
    imagesB = imagesB.cuda()
    imagesC = imagesC.cuda()

    begin_time = time.time()
    _, _, _, _, result_mcs = model(imagesA, imagesB, imagesC)
    outPRED_mcs = torch.cat((outPRED_mcs, result_mcs.data), 0)
    batch_time = time.time() - begin_time
    bar.suffix = '{} / {} | Time: {batch_time:.4f}'.format(epochID + 1, len(test_loader),
                                                           batch_time=batch_time * (iters_per_epoch - epochID) / 60)
    bar.next()
bar.finish()

# # save result into excel:
# save_output(label_test_file, outPRED_mcs, args, save_file=save_file_name)


Processing inference |                                | 8 / 3136 | Time: 3.209958

KeyboardInterrupt: 

In [ ]:
# /home/tomas/SYNC/BIO/EyeQ/MCF_Net/result/DenseNet121_v3_v1.csv
save_output_here(label_test_file, outPRED_mcs, save_file=save_file_name)
label_list = ['Good', 'Usable', 'Reject']
saver = SaveOutput(label_list)
saver.save(label_test_file, outPRED_mcs, save_file=save_file_name)


In [34]:
# evaluation:
df_gt = pd.read_csv(label_test_file)
img_list = df_gt["image"].tolist()
GT_QA_list = np.array(df_gt["quality"].tolist())
img_num = len(img_list)
label_list = ["Good", "Usable", "Reject"]

df_tmp = pd.read_csv(save_file_name)
predict_tmp = np.zeros([img_num, 3])
for idx in range(3):
    predict_tmp[:, idx] = np.array(df_tmp[label_list[idx]].tolist())
tmp_report = compute_metric(GT_QA_list, predict_tmp, target_names=label_list)

print(' Accuracy: ' + str("{:0.4f}".format(np.mean(tmp_report['Accuracy']))) +
      ' Precision: ' + str("{:0.4f}".format(np.mean(tmp_report['Precision']))) +
      ' Sensitivity: ' + str("{:0.4f}".format(np.mean(tmp_report['Sensitivity']))) +
      ' F1: ' + str("{:0.4f}".format(np.mean(tmp_report['F1']))))



calculator = MetricCalculator(GT_QA_list, predict_tmp, label_list)
calculator.get_metrics()

print(' Accuracy: ' + str("{:0.4f}".format(np.mean(calculator.metrics['Accuracy']))) +
      ' Precision: ' + str("{:0.4f}".format(np.mean(calculator.metrics['Precision']))) +
      ' Sensitivity: ' + str("{:0.4f}".format(np.mean(calculator.metrics['Sensitivity']))) +
      ' F1: ' + str("{:0.4f}".format(np.mean(calculator.metrics['F1']))))

 Accuracy: 0.9720 Precision: 0.9560 Sensitivity: 0.9570 F1: 0.9565
 Accuracy: 0.9720 Precision: 0.9560 Sensitivity: 0.9570 F1: 0.9565


In [ ]:
tmp_report

{'class_name': ['Good', 'Usable', 'Reject'],
 'F1': array([[0.98567747],
        [0.91285866],
        [0.97087379]]),
 'AUC': 0.9823343797523513,
 'Accuracy': 0.9720162640516623,
 'Sensitivity': array([[0.98526417],
        [0.91577825],
        [0.96982759]]),
 'Precision': array([[0.98609113],
        [0.90995763],
        [0.97192225]]),
 'Specificity': array([[0.97235462],
        [0.984063  ],
        [0.99364179]]),
 'ROC_curve': {'ROC_fpr_0': array([0.        , 0.00142993, 0.00405148, ..., 0.98021926, 0.98021926,
         1.        ]),
  'ROC_tpr_0': array([0.        , 0.16808434, 0.26404696, ..., 0.9998802 , 1.        ,
         1.        ]),
  'ROC_T_0': array([       inf, 0.98759127, 0.98759115, ..., 0.02327108, 0.02323476,
         0.01851259]),
  'AUC_0': 0.9881430916595163,
  'ROC_fpr_1': array([0.        , 0.        , 0.        , ..., 0.99896878, 0.99896878,
         1.        ]),
  'ROC_tpr_1': array([0.00000000e+00, 5.33049041e-04, 7.99573561e-03, ...,
         9.99466

In [17]:
data_train = DatasetGenerator(data_dir=train_images_dir, list_file=label_test_file, transform1=transform_list1, # label_train_file swapped with label_test_file
                              transform2=transformList2, n_class=n_classes, set_name='train')
train_loader = torch.utils.data.DataLoader(dataset=data_train, batch_size=batch_size,
                                               shuffle=True, num_workers=4, pin_memory=True)

In [18]:

data_val = DatasetGenerator(data_dir=train_images_dir, list_file=label_test_file, transform1=transform_list_val1,
                            transform2=transformList2, n_class=n_classes, set_name='val')
val_loader = torch.utils.data.DataLoader(dataset=data_val, batch_size=batch_size,
                                             shuffle=False, num_workers=4, pin_memory=True)

In [19]:
len(train_loader)
# get only 100 images for training
train_loader.dataset.image_names = train_loader.dataset.image_names[:100]
train_loader.dataset.labels = train_loader.dataset.labels[:100]

val_loader.dataset.image_names = val_loader.dataset.image_names[:100]
val_loader.dataset.labels = val_loader.dataset.labels[:100]
print('Train loader length: ', len(train_loader.dataset.image_names))
print('Train loader length: ', len(train_loader.dataset.labels))
print('Val loader length: ', len(val_loader.dataset.image_names))
print('Val loader length: ', len(val_loader.dataset.labels))
len(train_loader)


Train loader length:  100
Train loader length:  100
Val loader length:  100
Val loader length:  100


25

nejde validation, asi by to chcelo ist mimo loopu a postupne krokovat jednotlive kroky kedze validation je jedna velka zahada a neviem ako to funguje

+ jebly torch po sebe ale necistil pamet, takze to padalo na cuda out of memory, tak som musel restartovat kernel... tak tebe jebe copilot to proste vyriesil za mna ez

In [15]:
epochs = 20
loss_w = [0.1, 0.1, 0.1, 0.1, 0.6]
_ = train_step_here(train_loader, model, 0, optimizer, criterion)




iters_per_epoch:  25 




Processing train Epoch -> 1 / 20 |################################| 25 / 25 | Time: 0.00 mins | Loss: 0.0450 


In [47]:
epochs = 20
loss_w = [0.1, 0.1, 0.1, 0.1, 0.6]
trainer = Trainer(model, optimizer, criterion, loss_w, epochs)

In [48]:
trainer.train_step(train_loader, 0)



iters_per_epoch:  25 




Processing train Epoch -> 1 / 20 |################################| 25 / 25 | Time: 0.00 mins | Loss: 0.0530 


0.12453977305442095

In [16]:
train_loader.shape

AttributeError: 'DataLoader' object has no attribute 'shape'

In [49]:
# free all GPU memory:
torch.cuda.empty_cache()

# free all GPU memory different way then above:
import gc
gc.collect()
torch.cuda.empty_cache()





In [19]:

#val loader has only 3 values for each image, check if it is correct
(imagesA, imagesB, imagesC, labels) = val_loader.dataset.__getitem__(0)
print(imagesA.shape)
print(imagesB.shape)
print(imagesC.shape)
print(labels.shape)



torch.Size([3, 224, 224])
torch.Size([3, 224, 224])
torch.Size([3, 224, 224])
torch.Size([1, 3])


In [27]:
(imagesAa, imagesBa, imagesCa) = val_loader.dataset.__getitem__(0)
print(imagesAa.shape)
print(imagesBa.shape)
print(imagesCa.shape)
# get label for image 0
print(val_loader.dataset.labels[0].shape)
model.eval()
imagesAa = imagesAa.cuda()
imagesBa = imagesBa.cuda()
imagesCa = imagesCa.cuda()
labels = val_loader.dataset.labels[0]
# change from numpy to tensor
labels = torch.from_numpy(labels).float()
labels = labels.cuda()
labels = labels.squeeze(1)

_, _, _, _, outputs = model(imagesAa, imagesBa, imagesCa)
with torch.no_grad():
    loss = criterion(outputs, labels)
    print(loss.item())
    # epoch_loss += loss.item()


ValueError: too many values to unpack (expected 3)

In [33]:
epochs = 20
loss_w = [0.1, 0.1, 0.1, 0.1, 0.6]
# Train and val
for epoch in range(0, epochs):
    _ = train_step_here(train_loader, model, epoch, optimizer, criterion)
    validation_loss = validation_step_here(val_loader, model, criterion)
    print('Current Loss: {}| Best Loss: {} at epoch: {}'.format(validation_loss, best_metric, best_iter))

    # save model
    if best_metric > validation_loss:
        best_metric = validation_loss
        best_iter = epoch
        # model_save_file = os.path.join(args.save_dir, save_model + '.tar')
        # if not os.path.exists(save_dir):
        #     os.makedirs(save_dir)
        # torch.save({'state_dict': model.state_dict(), 'best_loss': best_metric}, model_save_file)
        # print('Model saved to %s' % model_save_file)




iters_per_epoch:  25 




Processing train Epoch -> 1 / 20 |################################| 25 / 25 | Time: 0.00 mins | Loss: 0.0517 


Current Loss: 0.033236809745430945| Best Loss: 0.024055843874812126 at epoch: 15


iters_per_epoch:  25 




Processing train Epoch -> 2 / 20 |################################| 25 / 25 | Time: 0.00 mins | Loss: 0.0306 


Current Loss: 0.029031456187367438| Best Loss: 0.024055843874812126 at epoch: 15


iters_per_epoch:  25 




Processing train Epoch -> 3 / 20 |#                               | 1 / 25 | Time: 0.08 mins | Loss: 0.0244 

KeyboardInterrupt: 

In [22]:
validation_loss = validation_step_here(val_loader, model, criterion)


In [23]:
validation_loss

0.05574897188693285

BCELoss()

In [13]:
def train_step_here(train_loader, model, epoch, optimizer, criterion):
    
    # switch to train mode
    model.train()
    epoch_loss = 0.0
    loss_w = [0.1, 0.1, 0.1, 0.1, 0.6]

    iters_per_epoch = len(train_loader)
    print('\n\niters_per_epoch: ', iters_per_epoch, '\n\n')
    bar = Bar('Processing {} Epoch -> {} / {}'.format('train', epoch+1, epochs), max=iters_per_epoch)
    bar.check_tty = False

    for step, (imagesA, imagesB, imagesC, labels) in enumerate(train_loader):
        start_time = time.time()

        torch.set_grad_enabled(True)

        imagesA = imagesA.cuda()
        imagesB = imagesB.cuda()
        imagesC = imagesC.cuda()

        labels = labels.cuda()
        # change the labels from shape [4,1,3] to [4,3]
        labels = labels.squeeze(1)

        out_A, out_B, out_C, out_F, combine = model(imagesA, imagesB, imagesC)

        loss_x = criterion(out_A, labels)
        loss_y = criterion(out_B, labels)
        loss_z = criterion(out_C, labels)
        loss_c = criterion(out_F, labels)
        loss_f = criterion(combine, labels)

        lossValue = loss_w[0]*loss_x+loss_w[1]*loss_y+loss_w[2]*loss_z+loss_w[3]*loss_c+loss_w[4]*loss_f


        optimizer.zero_grad()
        lossValue.backward()
        optimizer.step()

        # measure elapsed time
        epoch_loss += lossValue.item()
        end_time = time.time()
        batch_time = end_time - start_time
        # plot progress
        bar_str = '{} / {} | Time: {batch_time:.2f} mins | Loss: {loss:.4f} '
        bar.suffix = bar_str.format(step+1, iters_per_epoch, batch_time=batch_time*(iters_per_epoch-step)/60,
                                    loss=lossValue.item())
        bar.next()

    epoch_loss = epoch_loss / iters_per_epoch

    bar.finish()
    return epoch_loss

In [10]:
def validation_step_here(val_loader, model, criterion):

    # switch to train mode
    model.eval()
    epoch_loss = 0
    iters_per_epoch = len(val_loader)
    bar = Bar('Processing {}'.format('validation'), max=iters_per_epoch)

    for step, (imagesA, imagesB, imagesC, labels) in enumerate(val_loader):
        start_time = time.time()

        imagesA = imagesA.cuda()
        imagesB = imagesB.cuda()
        imagesC = imagesC.cuda()

        labels = labels.cuda()
        labels = labels.squeeze(1)

        _, _, _, _, outputs = model(imagesA, imagesB, imagesC)
        with torch.no_grad():
            loss = criterion(outputs, labels)
            epoch_loss += loss.item()

        end_time = time.time()

        # measure elapsed time
        batch_time = end_time - start_time
        bar_str = '{} / {} | Time: {batch_time:.2f} mins'
        bar.suffix = bar_str.format(step + 1, len(val_loader), batch_time=batch_time * (iters_per_epoch - step) / 60)
        bar.next()

    epoch_loss = epoch_loss / iters_per_epoch
    bar.finish()
    return epoch_loss

validation

In [41]:
validator = Validator(model, criterion)
validation_loss = validator.validate(val_loader)
validation_loss

0.05574897188693285